# MLiM: Purchase Prediction - Recommender System

* predict purchase probabilities for all customer-product combinations
* time period for making predictions is week 90
* 2000 shoppers and 250 products -> 500k predictions 

$\to$ for initial modeling: only 100 shoppers in week 1-20 (train) and week 21 (test) 

In [1]:
import os
import tqdm
import warnings
import functools

import numpy as np
import pandas as pd

import seaborn as sns
import sklearn.preprocessing
import sklearn.neighbors
import sklearn.model_selection
import sklearn.metrics

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

# import pyarrow

In [2]:
%cd C:/Users/Christopher/Documents/MLiM/Assignment 3/
os.getcwd()

C:\Users\Christopher\Documents\MLiM\Assignment 3


'C:\\Users\\Christopher\\Documents\\MLiM\\Assignment 3'

In [3]:
from statsmodels.graphics.tsaplots import plot_acf

In [4]:
#plot_acf(data['Count'], lags=10)
#plot_pacf(data['Count'], lags=10)

In [4]:
baskets = pd.read_parquet("baskets.parquet")

In [6]:
baskets.head() # 68.8mm products # 99999 shoppers # week 0-89

,week,shopper,product,price
0,0,0,71,629
1,0,0,91,605
2,0,0,116,715
3,0,0,123,483
4,0,0,157,592


In [7]:
pred_idx = pd.read_parquet("prediction_index.parquet")

In [8]:
pred_idx.head() # 2000 shopppers: 0-1999, 500.000 shopper-product combinations (predictions)

,week,shopper,product
0,90,0,0
1,90,0,1
2,90,0,2
3,90,0,3
4,90,0,4


In [9]:
coupons = pd.read_parquet("coupons.parquet")

In [10]:
coupons.head() # 45mm products discounted # 99999 shoppers # week 0-89

,week,shopper,product,discount
0,0,0,35,35
1,0,0,193,40
2,0,0,27,30
3,0,0,177,35
4,0,0,5,30


In [11]:
data = pd.merge(baskets, coupons, on=['week','shopper','product'], how='outer')
#after group discussion: changed left join to outer in order to get the information which products the shopper did NOT 
#buy and which products were NOT bought even though the shopper was offered a discount --> might indicate that the
#shopper dislikes some products very much --> low to none probability of buying it in week 90 
data.head()

,week,shopper,product,price,discount
0,0,0,71,629.0,NaN
1,0,0,91,605.0,NaN
2,0,0,116,715.0,NaN
3,0,0,123,483.0,NaN
4,0,0,157,592.0,NaN


In [12]:
data['discount'] = data['discount'].replace(np.nan, 0)

In [13]:
#discount offered to the shopper
data["discount_offered"] = np.where(data.discount != 0, 1, 0)
#product purchased
data["product_bought"] = np.where(data.price.isna(), 0, 1)
#purchase without having a discount
data["purchase_w/o_dis"] = np.where(((data.product_bought == 1) & (data.discount_offered == 0)), 1, 0)
#no purchase even though a discount was offered
data["no_purchase_w_dis"] = np.where(((data.product_bought == 0) & (data.discount_offered == 1)), 1, 0)

In [14]:
# @Asmir: moved the code here and renamed the variable (before: discounted) --> we had the same variable, just two names
data.discount_offered.value_counts()/len(data)

0    0.595082
1    0.404918
Name: discount_offered, dtype: float64

In [15]:
data.head(10)

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis
0,0,0,71,629.0,0.0,0,1,1,0
1,0,0,91,605.0,0.0,0,1,1,0
2,0,0,116,715.0,0.0,0,1,1,0
3,0,0,123,483.0,0.0,0,1,1,0
4,0,0,157,592.0,0.0,0,1,1,0
5,0,0,167,582.0,0.0,0,1,1,0
6,0,0,171,639.0,0.0,0,1,1,0
7,0,0,184,651.0,0.0,0,1,1,0
8,0,0,207,410.0,0.0,0,1,1,0
9,0,0,225,602.0,0.0,0,1,1,0


In [16]:
#discount effect --> either neutral (if shopper would have bought the item anyways) or positive 
data["discount_effect"] = np.where(((data.discount_offered == 1) & (data.product_bought == 1)), 1, 0)
data[(data.discount_effect == 1) & (data.week != 0)].head()

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect
764849,1,5,202,326.0,35.0,1,1,0,0,1
764867,1,7,114,405.0,30.0,1,1,0,0,1
764869,1,7,188,316.0,40.0,1,1,0,0,1
764885,1,9,120,415.0,35.0,1,1,0,0,1
764890,1,9,225,481.0,20.0,1,1,0,0,1


In [17]:
#discount might have an effect if shopper does not know product or has not purchased it before (see e.g. product 221
#or 202 for shopper 5)
#it does not have an effect if the shopper just bought a product or rather if the discount is not offered in the 
#shoppers "buying rhythm", see discount in week 68 (shopper 5 just bought the product 221 in week 67)

data[(data["product"] == 221) & (data["shopper"] == 5)]

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect
40,0,5,221,442.0,35.0,1,1,0,0,1
17596820,23,5,221,680.0,0.0,0,1,1,0,0
22186727,29,5,221,680.0,0.0,0,1,1,0,0
42070366,55,5,221,680.0,0.0,0,1,1,0,0
51249934,67,5,221,680.0,0.0,0,1,1,0,0
55072323,72,5,221,680.0,0.0,0,1,1,0,0
100795587,68,5,221,NaN,10.0,1,0,0,1,0


## @**ALL**: we set the max. shoppers to 100 in order to reduce the size of the data set

In [18]:
data_2 = data[data['shopper'] < 100]
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110869 entries, 0 to 110664311
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   week               110869 non-null  int64  
 1   shopper            110869 non-null  int64  
 2   product            110869 non-null  int64  
 3   price              68660 non-null   float64
 4   discount           110869 non-null  float64
 5   discount_offered   110869 non-null  int32  
 6   product_bought     110869 non-null  int32  
 7   purchase_w/o_dis   110869 non-null  int32  
 8   no_purchase_w_dis  110869 non-null  int32  
 9   discount_effect    110869 non-null  int32  
dtypes: float64(2), int32(5), int64(3)
memory usage: 7.2 MB


In [19]:
data_2.head() 

# @ASMIR: 1.38mm was for 2000 shoppers and without discount df

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect
0,0,0,71,629.0,0.0,0,1,1,0,0
1,0,0,91,605.0,0.0,0,1,1,0,0
2,0,0,116,715.0,0.0,0,1,1,0,0
3,0,0,123,483.0,0.0,0,1,1,0,0
4,0,0,157,592.0,0.0,0,1,1,0,0


In [20]:
#clearing memory 
del baskets
del coupons
del data

In [21]:
data_2[data_2.shopper == 0]

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect
0,0,0,71,629.0,0.0,0,1,1,0,0
1,0,0,91,605.0,0.0,0,1,1,0,0
2,0,0,116,715.0,0.0,0,1,1,0,0
3,0,0,123,483.0,0.0,0,1,1,0,0
4,0,0,157,592.0,0.0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
110193874,88,0,69,NaN,30.0,1,0,0,1,0
110193875,88,0,116,NaN,15.0,1,0,0,1,0
110663845,89,0,131,NaN,30.0,1,0,0,1,0
110663846,89,0,16,NaN,25.0,1,0,0,1,0


In [22]:
data_2[(data_2.shopper == 0) & (data_2.product_bought == 1)]['product'].value_counts() #54

91     73
225    68
71     67
130    61
184    52
123    43
202    39
67     35
167    32
157    30
116    30
171    29
87     22
76     17
238    14
249    13
211    12
210    11
214    10
52      9
191     9
11      9
179     8
4       8
244     7
207     6
110     6
38      5
148     5
16      4
56      3
181     3
81      3
80      2
100     2
98      2
79      2
172     2
48      2
37      1
185     1
138     1
21      1
122     1
119     1
162     1
144     1
224     1
59      1
232     1
15      1
240     1
243     1
46      1
Name: product, dtype: int64

In [23]:
data_2[(data_2.shopper == 0) & (data_2.product_bought == 1)]['week'].value_counts() # 90 weeks 

51    13
30    12
26    12
38    12
40    12
      ..
87     6
71     5
63     5
14     3
24     3
Name: week, Length: 90, dtype: int64

In [24]:
# product-price dict -> get prices for products 
#product_price_dict = data_2.set_index('product').to_dict()['price'] 
#data_3['price']= data_3['product'].map(product_price_dict)

# @Asmir: I did the price mapping with an aggregation function and left-joined the max and min price to the df (see the following code)

In [25]:
#maximal price of product
max_price = data_2.groupby("product")["price"].agg(max).reset_index()
max_price = max_price.rename(columns = {"price": "max_price"})

In [26]:
#merge max price to the df
data_2 = pd.merge(data_2, max_price, on = "product", how = "left")

#impute missing prices (of the coupon data set) by the max price minues the offered discount (because this was the 
#price the shoppers was offered)
data_2["price"] = np.where(data_2.price.isna(), (data_2.max_price*(100-data_2.discount)*0.01), data_2.price)
data_2

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect,max_price
0,0,0,71,629.00,0.0,0,1,1,0,0,629.0
1,0,0,91,605.00,0.0,0,1,1,0,0,605.0
2,0,0,116,715.00,0.0,0,1,1,0,0,715.0
3,0,0,123,483.00,0.0,0,1,1,0,0,483.0
4,0,0,157,592.00,0.0,0,1,1,0,0,592.0
...,...,...,...,...,...,...,...,...,...,...,...
110864,89,99,174,527.10,30.0,1,0,0,1,0,753.0
110865,89,99,46,392.25,25.0,1,0,0,1,0,523.0
110866,89,99,225,361.20,40.0,1,0,0,1,0,602.0
110867,89,99,127,464.25,25.0,1,0,0,1,0,619.0


In [27]:
#minimal price of product
min_price = data_2.groupby("product")["price"].agg(min).reset_index()
min_price = min_price.rename(columns = {"price": "min_price"})

data_2 = pd.merge(data_2, min_price, on = "product", how = "left")

In [28]:
lop = data_2[["shopper","week","product"]].sort_values(by=["shopper", "week"]).groupby(by=["shopper","week"]).agg({'product':lambda x: list(x)}).reset_index(drop=False)
lop = lop.rename(columns={"product": "list_of_products"})
# remove duplicates
lop["list_of_products"] = lop["list_of_products"].apply(lambda x: list(dict.fromkeys(x)))
lop.head()

,shopper,week,list_of_products
0,0,0,"[71, 91, 116, 123, 157, 167, 171, 184, 207, 22..."
1,0,1,"[71, 91, 130, 157, 167, 191, 225, 137, 92, 56,..."
2,0,2,"[52, 67, 71, 87, 91, 130, 184, 211, 238, 7, 17..."
3,0,3,"[71, 91, 123, 130, 179, 184, 214, 225, 154, 16..."
4,0,4,"[48, 67, 76, 91, 116, 123, 184, 225, 66, 105, ..."


In [29]:
# @Asmir: the var is not in this code document; did you save it somewhere else?
data_2['week_customer_product_sales'].value_counts()

KeyError: 'week_customer_product_sales'

In [30]:
data_2

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect,max_price,min_price
0,0,0,71,629.00,0.0,0,1,1,0,0,629.0,377.0
1,0,0,91,605.00,0.0,0,1,1,0,0,605.0,363.0
2,0,0,116,715.00,0.0,0,1,1,0,0,715.0,429.0
3,0,0,123,483.00,0.0,0,1,1,0,0,483.0,289.8
4,0,0,157,592.00,0.0,0,1,1,0,0,592.0,355.0
...,...,...,...,...,...,...,...,...,...,...,...,...
110864,89,99,174,527.10,30.0,1,0,0,1,0,753.0,451.8
110865,89,99,46,392.25,25.0,1,0,0,1,0,523.0,313.0
110866,89,99,225,361.20,40.0,1,0,0,1,0,602.0,361.2
110867,89,99,127,464.25,25.0,1,0,0,1,0,619.0,371.0


In [160]:
data_2 = data_2[["week","shopper", "product", "price", "discount", "discount_offered", "product_bought", "purchase_w/o_dis", "no_purchase_w_dis", "discount_effect", "max_price", "min_price"]]

In [161]:
tmp = data_2[data_2.product_bought == 1].groupby(['shopper'])['product'].agg('count').reset_index().rename(columns={"product":"no_products_bought"})
data_2 = pd.merge(data_2, tmp, on="shopper", how="left")


tmp = data_2[data_2.product_bought == 1].groupby(['shopper'])['price'].agg('sum').reset_index().rename(columns={"price":"spend"})
data_2 = pd.merge(data_2, tmp, on="shopper", how="left")

data_2['mean_product_price'] = data_2.spend/data_2.no_products_bought


tmp = data_2[data_2.product_bought == 1].groupby(['shopper','product'])['discount_offered'].agg('mean').reset_index().rename(columns={"discount_offered":"customer_product_discounted_share"})
data_2 = pd.merge(data_2, tmp, on=['shopper','product'], how="left")

tmp = data_2[data_2.product_bought == 1].groupby(['shopper','product'])['price'].agg('count').reset_index().rename(columns={"price":"customer_product_sales"})
data_2 = pd.merge(data_2, tmp, on=['shopper','product'], how="left")
data_2[data_2.customer_product_sales.isna()] = 0 # never bought

tmp = data_2[data_2.product_bought == 1].groupby(['shopper'])['week'].agg('nunique').reset_index().rename(columns={"week":"no_shopping_events"})
data_2 = pd.merge(data_2, tmp, on="shopper", how="left")


tmp = data_2[data_2.product_bought == 1].groupby(['week','shopper'])['product'].agg('count').reset_index().rename(columns={"product":"week_basket_size"})
data_2 = pd.merge(data_2, tmp, on=['week','shopper'], how="left")

tmp = data_2[data_2.product_bought == 1].groupby(['week','shopper'])['price'].agg('sum').reset_index().rename(columns={"price":"week_basket_value"})
data_2 = pd.merge(data_2, tmp, on=['week','shopper'], how="left")

tmp = data_2[data_2.product_bought == 1].groupby(['week', 'shopper', 'product'])['price'].agg('count').reset_index().rename(columns={"price":"week_customer_product_sales"})
data_2 = pd.merge(data_2, tmp, on=['week', 'shopper', 'product'], how="left")
data_2[data_2.week_customer_product_sales.isna()] = 0 # nothing bought

data_2["no_products_bought"] = data_2["no_products_bought"] + 1 # avoid div/0
data_2['mean_customer_product_sales'] = data_2.customer_product_sales/data_2.no_products_bought 

tmp = data_2[data_2.product_bought == 1].groupby(['shopper'])['week_basket_size'].agg('mean').reset_index().rename(columns={"week_basket_size":"mean_basket_size"})
data_2 = pd.merge(data_2, tmp, on="shopper", how="left")

tmp = data_2[data_2.product_bought == 1].groupby(['shopper'])['week_basket_value'].agg('mean').reset_index().rename(columns={"week_basket_value":"mean_basket_value"})
data_2 = pd.merge(data_2, tmp, on="shopper", how="left")

#average offered discount per week or rather average coupon value per shopper
tmp = data_2.groupby("week")["discount"].agg("mean").reset_index().rename(columns={"discount":"ave_offered_dis_week"})
data_2 = pd.merge(data_2, tmp, on="week", how="left")
#average used discount per week
tmp = data_2[data_2.product_bought == 1].groupby("week")["discount"].agg("mean").reset_index().rename(columns={"discount":"ave_used_dis_week"})
data_2 = pd.merge(data_2, tmp, on="week", how="left")
data_2

,week,shopper,product,price,discount,discount_offered,product_bought,purchase_w/o_dis,no_purchase_w_dis,discount_effect,...,customer_product_sales,no_shopping_events,week_basket_size,week_basket_value,week_customer_product_sales,mean_customer_product_sales,mean_basket_size,mean_basket_value,ave_offered_dis_week,ave_used_dis_week
0,0,0,71,629.0,0.0,0,1,1,0,0,...,67.0,90,10,5908.0,1.0,0.086900,8.992208,5273.981818,0.017695,1.025641
1,0,0,91,605.0,0.0,0,1,1,0,0,...,73.0,90,10,5908.0,1.0,0.094682,8.992208,5273.981818,0.017695,1.025641
2,0,0,116,715.0,0.0,0,1,1,0,0,...,30.0,90,10,5908.0,1.0,0.038911,8.992208,5273.981818,0.017695,1.025641
3,0,0,123,483.0,0.0,0,1,1,0,0,...,43.0,90,10,5908.0,1.0,0.055772,8.992208,5273.981818,0.017695,1.025641
4,0,0,157,592.0,0.0,0,1,1,0,0,...,30.0,90,10,5908.0,1.0,0.038911,8.992208,5273.981818,0.017695,1.025641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110864,0,0,0,0.0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0.000000,8.992208,5273.981818,0.017695,1.025641
110865,0,0,0,0.0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0.000000,8.992208,5273.981818,0.017695,1.025641
110866,0,0,0,0.0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0.000000,8.992208,5273.981818,0.017695,1.025641
110867,0,0,0,0.0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0.000000,8.992208,5273.981818,0.017695,1.025641


In [162]:
data_2.isna().sum()

week                                 0
shopper                              0
product                              0
price                                0
discount                             0
discount_offered                     0
product_bought                       0
purchase_w/o_dis                     0
no_purchase_w_dis                    0
discount_effect                      0
max_price                            0
min_price                            0
no_products_bought                   0
spend                                0
mean_product_price                   0
customer_product_discounted_share    0
customer_product_sales               0
no_shopping_events                   0
week_basket_size                     0
week_basket_value                    0
week_customer_product_sales          0
mean_customer_product_sales          0
mean_basket_size                     0
mean_basket_value                    0
ave_offered_dis_week                 0
ave_used_dis_week        

In [33]:
data_2.isna().sum()

week                                     0
shopper                                  0
product                                  0
price                                    0
discount                                 0
discount_offered                         0
product_bought                           0
purchase_w/o_dis                         0
no_purchase_w_dis                        0
discount_effect                          0
max_price                                0
min_price                                0
no_products_bought                   42209
spend                                42209
mean_product_price                   42209
customer_product_discounted_share    42209
customer_product_sales               42209
no_shopping_events                   42209
week_basket_size                     42209
week_basket_value                    42209
week_customer_product_sales          42209
mean_customer_product_sales          42209
mean_basket_size                     42209
mean_basket

In [164]:
data_2.sort_values(by=["week","shopper"], inplace=True)
data_2.to_parquet('data_100.parquet')

## @**ALL**: Impute NaNs

One idea is to make a separate df (here called data_shopper) with only the basket data, engineer all features, merge with a left join back to data_2; somehow is does not work and my machine crushes everytime - any idea why? See code below

Any other ideas?

In [ ]:
data_shopper = data_2.loc[data_2.product_bought == 1, ["shopper", "week", "product", "price", "discount", "discount_offered"]]

data_shopper['no_products_bought'] = data_shopper.groupby(['shopper'])['product'].transform('count')
data_shopper['spend'] = data_shopper.groupby(['shopper'])['price'].transform('sum')
data_shopper['mean_product_price'] = data_shopper.spend/data_shopper.no_products_bought
data_shopper['customer_product_discounted_share'] = data_shopper.groupby(['shopper','product'])['discount_offered'].transform('mean')
data_shopper['customer_product_sales'] = data_shopper.groupby(['shopper','product'])['product'].transform('count')
data_shopper['no_shopping_events'] = data_shopper.groupby(['shopper'])['week'].transform('nunique')

data_shopper['week_basket_size'] = data_shopper.groupby(['week','shopper'])['product'].transform('count')
data_shopper['week_basket_value'] = data_shopper.groupby(['week','shopper'])['price'].transform('sum')
data_shopper['week_customer_product_sales'] = data_shopper.groupby(['week', 'shopper', 'product'])['price'].transform('count')
data_shopper['mean_customer_product_sales'] = data_shopper['customer_product_sales']/data_shopper['no_products_bought']

data_shopper['mean_basket_size'] = data_shopper.groupby(['shopper'])['week_basket_size'].transform('mean')
data_shopper['mean_basket_value'] = data_shopper.groupby(['shopper'])['week_basket_value'].transform('mean')

#average coupon value per shopper
data_shopper["ave_discount_offered"] = data_shopper[data_shopper.discount_offered == 1].groupby("shopper")["discount"].transform("mean")
data_shopper

## Feature ideas

* products_count: data_2.groupby(['shopper'])['product'].transform.('count')
* basket_size: data_2.groupby(['week','shopper'])['product'].transform.('count')
* mean_basket_size: data_2.groupby(['shopper'])['basket_size'].transform.('mean')
---
* spend: data_2.groupby(['shopper'])['price'].transform.('sum')
* basket_value: data_2.groupby(['week','shopper'])['price'].transform.('sum')
* mean_basket_value: data_2.groupby(['shopper'])['basket_value'].transform.('mean') or mean_basket_value: data_2.spend/data_2.products_count
---
* discounted: data_2["discounted"] = np.where(data_2["discount"] > 0,1,0)
* customer_product_discounted_share: #How often did this customer buy this product at a discounted price? 
---
* how often did customer buy that product? 
* how often did customer buy that product at discounted price?
* how many items of that product did customer buy at specific week? 
* what share of all products bought is that product for customer? 

## DONE!

In [ ]:
data_2['Last_Week_Sales'] = data_2.groupby(['shopper', 'product'])['Sales'].shift()

## TO DO !
--- 
### Creating lag features considering the customer-product combination
* how many weeks since customer bought product the last time, the 2nd last time, 3rd last time, 4th last time? 
    * lag_product_0, lag_product_1, lag_product_2, lag_product_3
* how many weeks on average between buying that product? 
---
* Does shopper buy every week? How many weeks does shopper shop?  At which weeks? 
* 
---
* Demand?/ how often is that product bought on average (whole population) 
* Customer_Demand vs. General_Demand
* 
---
* price elasticity for customer
* price elasticity for customer for that specific product? 
* price elasticity for specific product > price elasticity for customer?
---
* demand for product?
---
* category? 
* is product brought along with complementary product? 
* is product substitued for substitute product? 
* association rules mining/ co-occurance matrix

In [ ]:
# Bracket the weeks
max_week = data_2['week'].max()  
min_week = data_2['week'].min()  
weeks = (max_week - min_week)

# Master list of weeks
week_list = [min_week + datetime.timedelta(weeks=x) for x in range(weeks)]
week_list.sort(reverse=True)

# Master list of shoppers
unique_shoppers = list(set(data_2['shopper'].unique()))
unique_shoppers.sort()
num_unique_shoppers = len(unique_shoppers)

# Unique Week / Shopper index
week_index = np.repeat(weeks, num_unique_shoppers) # num weeks * num shoppers
shopper_index = np.concatenate([unique_shoppers]*days)

row_count = weeks * len(unique_shoppers)

train_test_master = pd.DataFrame(index=range(row_count), columns=all_cols)

train_test_master['week'] = date_index
train_test_master['shopper'] = store_index

# Set the indexes (makes it easy to insert test/train data into master)
train_test_master.set_index(['Date', 'Store'], drop=True, inplace=True)
train.set_index(['Date', 'Store'], drop=True, inplace=True)
test.set_index(['Date', 'Store'], drop=True, inplace=True)

# Update the master index with train and test
train_test_master.update(train)
train_test_master.update(test)

In [ ]:
data['rolling_mean'] = data['Count'].rolling(window=7).mean() # rolling window

![RollingWindowUrl](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/11/3hotmk.gif "Rolling Window")

In [ ]:
data_2['expanding_mean'] = data_2['Count'].expanding(2).mean() # expanding window 

![SegmentLocal](expanding_window.gif "expanding window")

In [110]:
df_lagged = data_2.copy()
trailing_window_size = 10

for window in range(1, trailing_window_size + 1):
    shifted = data_2.shift(window)
    shifted.columns = [x + "_lag" + str(window) for x in data_2.columns]
    
    df_lagged = pd.concat((df_lagged, shifted), axis=1)
df_lagged = df_lagged.dropna()
df_lagged.head()

,week,shopper,product,price,discount,no_products_bought,week_basket_size,mean_basket_size,spend,week_basket_value,...,mean_basket_size_lag10,spend_lag10,week_basket_value_lag10,mean_basekt_value_lag10,mean_product_price_lag10,discounted_lag10,customer_product_discounted_share_lag10,customer_product_sales_lag10,week_customer_product_sales_lag10,mean_customer_product_sales_lag10
10,0,1,22,528,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.029851,67.0,1.0,0.087013
11,0,1,35,602,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.041096,73.0,1.0,0.094805
12,0,1,92,543,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.033333,30.0,1.0,0.038961
13,0,1,146,596,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.000000,43.0,1.0,0.055844
14,0,1,168,588,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.000000,30.0,1.0,0.038961


In [ ]:
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) # this pandas method returns a copy of the df, with group columns assigned the key value
    return (df.sort_values(by=["date"], ascending=True)
        .set_index(["date"])
        .shift(1)
               ) # the parenthesis allow you to chain methods and avoid intermediate variable assignment


In [ ]:
dflist = [lag_by_group(g, grouped_df.get_group(g)) for g in grouped_df.groups.keys()]
pd.concat(dflist, axis=0).reset_index())

In [124]:
#import numpy as np

def init_nans(df, num_periods, lag_vars, col_name_suf):
    '''
    Initiate column(s) of nan values for your lag columns in the given dataframe
    Inputs:
        df - dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    '''
    for period in range(1, num_periods + 1):
        for lag_var in lag_vars:
            df['{}_lag{}{}'.format(lag_var, period, col_name_suf)] = np.nan

def set_lag_vals(df, comb_mask, date_col, lag_vars, num_periods, col_name_suf):
    '''
    Sets lag values according to the last column in the heirarchy or columns
    Inputs:
        df - dataframe
        comb_mask - combined mask for current value combinations between columns. Used to filter dataframe
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    '''
    foo = df[ comb_mask ].sort_values(date_col, ascending=False)     
    length = len(foo[date_col].unique()) # determine number of visits (because multiple item categories can be updated in a single visit)
    for period in range(1, num_periods + 1):
        # skip if there's not enough data to create lag columns
        if length < period + 1:
            continue
        i = 0
        # create duplicate df, but with all indices shifted by the current 'period' number
        foo_shifted = foo.shift(-period)
        foo_grouped = foo.groupby(date_col).mean()
        for index, row in foo.iterrows():
            date = foo_shifted[ foo_shifted.index == index ][date_col].values[0]
            for lag_var in lag_vars:
                lag_val = foo_grouped[ foo_grouped.index == date ][lag_var].values[0]
                # set value
                df.set_value(index, '{}_lag{}{}'.format(lag_var, period, col_name_suf), lag_val)
            i += 1
            if i + period == length:
                break # back to period loop

def lag_rec(df, num_periods, col_filters, date_col, lag_vars, col_name_suf, mask=True):
    '''
    Recursively loop through various heirarchaly ordered columns, grouping by date
    INPUTS:
        df - pandas dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_filters - columns to heiracrchally filter down on, with the last column being the one ultimately used
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
        mask - DO NOT CHANGE. Required to be True to maintain dataframe mask between recursive iterations
    '''
    # begin with mask of all trues
    true_mask = np.ones(len(df), dtype=bool)
    loop_mask = mask & true_mask
    col_filter = col_filters[0]
    for val in df[ loop_mask ][col_filter].unique():
        val_mask = df[col_filter] == val
        comb_mask = loop_mask & val_mask

        if len(col_filters) > 1:
            #recursively update the remaining items' positions
            lag_rec(df, num_periods, col_filters[1:], date_col, lag_vars, col_name_suf, mask=comb_mask)
        else:
            set_lag_vals(df, comb_mask, date_col, lag_vars, num_periods, col_name_suf)

def lag(df, num_periods, col_filters, date_col, lag_vars, col_name_suf):
    '''
    Wrapper function to execute init_nans and lag_rec.
    Inputs:
        df - dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_filters - columns to heiracrchally filter down on, with the last column being the one ultimately used
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    Returns:
        Updated dataframe containing new lag columns
    '''
    init_nans(df, num_periods, lag_vars, col_name_suf)
    lag_rec(df, num_periods, col_filters, date_filter, lag_vars, col_name_suf)
    return df

# Problems

##### Products are not in basket every week. But pred_idx has all products listed for week 90 for each shopper
* What to do with products which are not bought each week? 
    * Build training data set including products which are not bougth 
    * Build feature "sold" (1/0) 
    * Derive product price from list 

## Train-Test-Splitting (for Time Series Data)

1. Train-Test-Split
- respects temporal order of observations
- useful for large amounts of data in order to make the train and test split representative of the original problem

2. Multiple Train-Test-Split
- respect temporal order of observations
- makes results more robust
- increase computational expense
- rather than repeating 1. manually, use scikit-learn library [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)
    - the train set varies
    - the test sample is kept constant during spliting
    - trained models can be combined and compared

3. Walk-Forward Validation
- model may be updated each time step new data is received moving along timeline
- hyperparameters are
    - min. no. of observations required to train the model $\to$ window width (if sliding window is used)
    - sliding or expanding window
        - expanding window: model is trained on all available data
        - sliding window: model is only trained on most recent data

source: https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/


## @**ALL**: Timeframe 1-20 for training and 21 for testing

We set the timeframe to week 1-20 for the training set and week 21 for testing (see co; the reason being is that the discount seems to have a little impact on the "first purchase"; we are afraid we are loosing this information if we kick out the earlier purchases; the week 0 is excluded because everything seems to be a little different in this week


In [44]:
#TRAIN-TEST-SPLIT TO USE

#train-test-split manually (train: week 1-20; test: 21)
data = pd.read_parquet("data_100.parquet")

train = data[(data.week <= 20) & (data.week > 0)]
test = data[(data.week == 21)]
print('Observations: %d' % (len(train) + len(test)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

train.to_parquet('train_100_1-20.parquet')
test.to_parquet('test_100_21.parquet')

Observations: 25838
Training Observations: 24612
Testing Observations: 1226


In [ ]:
#set final df! 

X = FINAL_DF_HERE.values

#further adaptions for 3. walk-forward-validation

    #counts how many obs. per timestep (1 timestep = 1 week)
#week_count = bask["week"].value_counts().reset_index().sort_values(by = "index", ascending = True) 

    #cummulated sum
#timestep = week_count["week"].cumsum(axis = 0).reset_index() 

    #cumulated sum of obs. per week
#t = timestep.week 

In [11]:
#train-test-split 2/3 (train) and 1/3 (test)
#train and test data sets are arrays after splitting

#X = bask.values
train_size = int(len(X) * 0.66)
train, test = X[0:train_size], X[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

Observations: 68841598
Training Observations: 45435454
Testing Observations: 23406144


In [7]:
#multiple train-test-split

from sklearn.model_selection import TimeSeriesSplit

#X = bask.values
splits = TimeSeriesSplit(n_splits=5)
index = 1
for train_index, test_index in splits.split(X):
	train = X[train_index]
	test = X[test_index]
	print('Observations: %d' % (len(train) + len(test)))
	print('Training Observations: %d' % (len(train)))
	print('Testing Observations: %d' % (len(test)))

Observations: 22947202
Training Observations: 11473603
Testing Observations: 11473599
Observations: 34420801
Training Observations: 22947202
Testing Observations: 11473599
Observations: 45894400
Training Observations: 34420801
Testing Observations: 11473599
Observations: 57367999
Training Observations: 45894400
Testing Observations: 11473599
Observations: 68841598
Training Observations: 57367999
Testing Observations: 11473599


In [10]:
#walk-forward validation with min. training set of 3 timesteps and expanding window
#from sklearn.model_selection import TimeSeriesSplit

#X = bask.values
#week_count = bask["week"].value_counts().reset_index().sort_values(by = "index", ascending = True)
#timestep = week_count["week"].cumsum(axis = 0).reset_index()
#t = timestep.week #cumulated sum of obs. per week

def walk_forward_val(min_train_timesteps, timesteps):
  min_train_timesteps = min_train_timesteps #min. timesteps of training set
  timestep = timesteps[min_train_timesteps-1:len(timesteps)-1]
  return timestep

j = 3 #min. training set = 3 timesteps
#t = timestep.week #cumulated sum of obs. per week
for i in walk_forward_val(j, t): 
    print("Size of i+1 timeframe (aka test data): ", t[j]-i)
    train, test = X[0:i], X[i:t[j]]
    print("train = %d, test = %d" % (len(train), len(test)))
    j += 1

Size of i+1 timeframe (aka test data):  763925
train = 2294602, test = 763925
Size of i+1 timeframe (aka test data):  765634
train = 3058527, test = 765634
Size of i+1 timeframe (aka test data):  764581
train = 3824161, test = 764581
Size of i+1 timeframe (aka test data):  765497
train = 4588742, test = 765497
Size of i+1 timeframe (aka test data):  764219
train = 5354239, test = 764219
Size of i+1 timeframe (aka test data):  766121
train = 6118458, test = 766121
Size of i+1 timeframe (aka test data):  765340
train = 6884579, test = 765340
Size of i+1 timeframe (aka test data):  765199
train = 7649919, test = 765199
Size of i+1 timeframe (aka test data):  764910
train = 8415118, test = 764910
Size of i+1 timeframe (aka test data):  764891
train = 9180028, test = 764891
Size of i+1 timeframe (aka test data):  765958
train = 9944919, test = 765958
Size of i+1 timeframe (aka test data):  766283
train = 10710877, test = 766283
Size of i+1 timeframe (aka test data):  764544
train = 11477160